In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import convolve

%matplotlib inline
%config InlineBackend.figure_format='retina'

## Local Polynomial Approximation

Set the parameters


In [ ]:
# maximum degree of polynomial used for fitting
N = 2

# filter size
M = 51

# large M, small N -> a lot of bias when frequency increases
# N = 2
# M = 51

# large M, large N -> smaller bias than before, but a lot of variance in smooth regions
# N = 7
# M = 51

# small M, small N -> smaller bias everywhere, higher variance
# N = 2
# M = 5

# M small, M = N,  -> smaller bias than before, higher variance than before
# N = 5
# N = M

Generate synthetic signal signal


In [ ]:
LENGHT = 1000

ty = np.linspace(0, 1, LENGHT)
y = np.sin(2 / (ty + 0.05))

#  noise standard deviation
sigma = 0.1

# noisy signal
s = y + sigma * np.random.normal(size=LENGHT)


In [ ]:
plt.figure()
plt.plot(ty, s, "r.")
plt.plot(ty, y, "k--", linewidth=2)
plt.grid()
plt.legend(["noisy", "original"])
plt.title("Input Signal")

Define the matrix T containing the polynomials sampled over the window


In [ ]:
t = np.arange(M) / (M - 1)  # normalized time over the window [0, 1]
T = np.zeros((M, N + 1))
for i in range(N + 1):
    T[:, i] = t**i

Look at the matrix T, the columns correspond to polynomials sampled over the interval [0,1]


In [ ]:
plt.figure()
leg = []
for l in range(N + 1):
    plt.plot(t, T[:, l])
    leg.append(f"t^{l:d}")
plt.legend(leg)

Construct the LPA kernel


In [ ]:
# comput the qr decomposition of WT
# since T has more rows than columns, then qr computes only the first N + 1 columns of Q and the first N + 1 rows of R.
Q, R = np.linalg.qr(T)

# select the central row of Q
row = int((M - 1) / 2)

# compute the kernel (first row of Q corresponds to the constant term)
g = Q[row, 0] * Q[:, 0]

# flipping, since it is used in convolution
g = np.flip(g)

In [ ]:
plt.figure()
plt.plot(g, "rs", linewidth=3)
plt.title(f"M = {M:d}, N = {N:d}")

Filtering


In [ ]:
shat = convolve(s, g, mode="same")

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(ty, s, "r.")
plt.plot(ty, y, "k--", linewidth=3)
plt.plot(ty, shat, "m-", linewidth=3)
plt.grid()
plt.legend(["noisy", "original", "LPA estimate"])
plt.title(f"M = {M:d}, N = {N:d}")


### Perform different M and N

- Large M, Large N (M=51, N=7)
- Small M, Small N (M=5, N=2)
- Small M, comparable N (M=7, N=5)


In [ ]:
# Parameter combinations to test
param_sets = [
    {"M": 51, "N": 7, "title": "Large M, Large N"},
    {"M": 5, "N": 2, "title": "Small M, Small N"},
    {"M": 7, "N": 5, "title": "Small M, comparable N"},
]

# Generate test signal (same as original)
LENGHT = 1000
ty = np.linspace(0, 1, LENGHT)
y = np.sin(2 / (ty + 0.05))
sigma = 0.1
s = y + sigma * np.random.normal(size=LENGHT)

# Test each parameter combination
for params in param_sets:
    M, N = params["M"], params["N"]

    # Define polynomial matrix T
    t = np.arange(M) / (M - 1)
    T = np.zeros((M, N + 1))
    for i in range(N + 1):
        T[:, i] = t**i

    # QR decomposition
    Q, R = np.linalg.qr(T)

    # Compute kernel
    row = int((M - 1) / 2)
    g = Q[row, 0] * Q[:, 0]
    g = np.flip(g)

    # Filter signal
    shat = convolve(s, g, mode="same")

    # Plot using your specified format
    plt.figure(figsize=(10, 10))
    plt.plot(ty, s, "r.")
    plt.plot(ty, y, "k--", linewidth=3)
    plt.plot(ty, shat, "m-", linewidth=3)
    plt.grid()
    plt.legend(["noisy", "original", "LPA estimate"])
    plt.title(f"M = {M:d}, N = {N:d} - {params['title']}")

    # Calculate and display MSE
    mse = np.mean((shat - y) ** 2)
    print(f"{params['title']} (M={M}, N={N}): MSE = {mse:.6f}")

plt.show()

## Weighted LPA

Set the parameters


In [ ]:
# maximum degree of polynomial used for fitting
N = 1

# filter size
M = 53

# half filter size
HFS = int((M - 1) / 2)

# set the weights. Here weights simply define the window size
w = np.zeros(M)

# centered kernel
wc = w.copy()
wc[int(HFS / 2) : -int(HFS / 2)] = 1

# left kernel
wl = w.copy()
wl[: HFS + 1] = 1

# right kernel
wr = w.copy()
wr[-HFS - 1 :] = 1

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].plot(wc, "rs", linewidth=3)
ax[0].set_title("centered weights")
ax[0].grid()
ax[1].plot(wl, "bs", linewidth=3)
ax[1].set_title("left weights")
ax[1].grid()
ax[2].plot(wr, "ms", linewidth=3)
ax[2].set_title("right weights")
ax[2].grid()


Build the weight matrix


In [ ]:
# select a single weight to be used in what follows
w = wc

# generate the inverse of weights
winv = np.zeros_like(w)
winv[w != 0] = 1 / w[w != 0]

# set to zero weights that are inf
winv[np.isinf(winv)] = 0

# define the weight matrix
W = np.diag(w)
Winv = np.diag(winv)

Generate synthetic signal signal


In [ ]:
LENGHT = 1000

# clean signal
ty = np.linspace(0, 1, LENGHT)
y = 8 * ty**2 - 2 * ty + 2
y[LENGHT // 2 : LENGHT] = y[LENGHT // 2 : LENGHT] + 7

#  noise standard deviation
sigma = 0.2

# noisy signal
s = y + sigma * np.random.normal(size=LENGHT)

In [ ]:
plt.figure()
plt.plot(ty, s, "r.")
plt.plot(ty, y, "k--", linewidth=2)
plt.grid()
plt.legend(["noisy", "original"])
plt.title("Input Signal")

Define the matrix T containing the polynomials sampled over the window


In [ ]:
t = np.arange(M) / (M - 1)  # normalized time over the window [0, 1]
T = np.zeros((M, N + 1))
for i in range(N + 1):
    T[:, i] = t**i

Construct the LPA kernel


In [ ]:
# comput the qr decomposition of WT
# since T has more rows than columns, then qr computes only the first N + 1 columns of Q and the first N + 1 rows of R.
Q, R = np.linalg.qr(W @ T)

#  define Qtilde
Qtilde = Winv @ Q

# adjust Qtilde with the  weights matrix squared.
W2Qtilde = W @ W @ Qtilde

# select the central row of W2Qtilde
row = int((M - 1) / 2)

# compute the kernel
g = W2Qtilde[row, 0] * Qtilde[:, 0]

# flipping, since it is used in convolution
g = np.flip(g)

In [ ]:
plt.figure()
plt.plot(g, "rs", linewidth=3)
plt.title(f"filter M = {M:d}, N = {N:d}")


Filtering


In [ ]:
shat = convolve(s, g, mode="same")

In [ ]:
plt.figure()
plt.plot(ty, s, "r.")
plt.plot(ty, y, "k--", linewidth=3)
plt.plot(ty, shat, "m-", linewidth=3)
plt.grid()
plt.legend(["noisy", "original", "LPA estimate"])
plt.title(f"M = {M:d}, N = {N:d}")


Modify the code to use the central, left and right kernels


In [ ]:
def adaptive_lpa_filter(signal, M, N, weights_list):
    filtered_signals = []

    for w in weights_list:
        # generate the inverse of weights
        winv = np.zeros_like(w)
        winv[w != 0] = 1 / w[w != 0]
        winv[np.isinf(winv)] = 0

        # define the weight matrices
        W = np.diag(w)
        Winv = np.diag(winv)

        # Define the matrix T
        t = np.arange(M) / (M - 1)
        T = np.zeros((M, N + 1))
        for i in range(N + 1):
            T[:, i] = t**i

        # QR decomposition
        Q, R = np.linalg.qr(W @ T)

        # Compute kernel
        Qtilde = Winv @ Q
        W2Qtilde = W @ W @ Qtilde
        row = int((M - 1) / 2)
        g = W2Qtilde[row, 0] * Qtilde[:, 0]
        g = np.flip(g)

        # Filter the signal
        filtered = convolve(signal, g, mode="same")
        filtered_signals.append(filtered)

    return filtered_signals


# Apply adaptive filtering with all three kernels
filtered_results = adaptive_lpa_filter(s, M, N, [wl, wc, wr])

# Plot results with different kernels
plt.figure(figsize=(12, 8))
plt.plot(ty, s, "r.", alpha=0.5, markersize=2)
plt.plot(ty, y, "k--", linewidth=3)
plt.plot(ty, filtered_results[0], "b-", linewidth=2, alpha=0.7)
plt.plot(ty, filtered_results[1], "m-", linewidth=2, alpha=0.7)
plt.plot(ty, filtered_results[2], "g-", linewidth=2, alpha=0.7)
plt.grid()
plt.legend(["noisy", "original", "left kernel", "centered kernel", "right kernel"])
plt.title(f"Adaptive LPA with different kernels (M = {M:d}, N = {N:d})")
plt.xlabel("Time")
plt.ylabel("Signal")

plt.show()